# Computing scores for a list of selected zonation markers in order to define PP and PC spots

Based on the results presentedo on previous scripts, we selected a list of periportal (PP) and pericentral (PC) markers. We will use these genes to compute a per spot score that will help us to assign the spots to different regions. 

In [ ]:
import os
import anndata as ad
import numpy as np
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import distance

from wrapper_functions import *

In [ ]:
# Automatically re-load wrapper functions after an update
# Find details here: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
organism = Organism.mouse
analyze_params = Analyze(protocol=Protocol.FF, organism=organism)

In [ ]:
root_path = os.getcwd()
inpath='your_inpath_folder' # Replace with the location of your samples
results_folder = os.path.join(root_path, 'analyzed')

In [ ]:
file_names = [f for f in os.listdir(results_folder) if os.path.isfile(os.path.join(results_folder, f))]

adata_list = [ad.read(os.path.join(results_folder, file)) for file in file_names if file.endswith('.h5ad')]

In [ ]:
Pericentral_markers = ['Glul','Cyp2e1','Oat','Slc1a2','Cyp1a2']
Periportal_markers = ['Sds', 'Cyp2f2', 'Hal', 'Hsd17b13', 'Alb', 'Arg1', 'Pck1']

In [ ]:
for adata in adata_list:
    
    # We store the raw counts into the layers attribute for further usage. 
    adata.X = np.round(adata.X)
    adata.layers['counts'] = adata.X.copy()

    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000, inplace=True)
    
    adata.layers['normalized'] = adata.X
    
    sc.tl.score_genes(adata, gene_list=Pericentral_markers, score_name='Pericentral_Score')
    sc.tl.score_genes(adata, gene_list=Periportal_markers, score_name='Periportal_Score')
    
    print(adata.obs['Sample_ID'].unique()[0])
    print(adata.obs['Condition'].unique()[0])
    print(adata.obs['Gender'].unique()[0])
    
    sc.pl.spatial(adata, color='Pericentral_Score')
    sc.pl.spatial(adata, color='Periportal_Score')
    
    threshold_pericentral = np.percentile(adata.obs['Pericentral_Score'], 80)  # 90th percentile as threshold
    threshold_periportal = np.percentile(adata.obs['Periportal_Score'], 80)  # 90th percentile as threshold
    adata.obs['zonation'] = "Other"
    
    print(f"# threshold_pericentral: {threshold_pericentral}")
    print(f"#threshold_periportal : {threshold_periportal}")
    
    adata.obs.loc[adata.obs['Pericentral_Score'] > threshold_pericentral, 'zonation'] = 'Pericentral'
    adata.obs.loc[adata.obs['Periportal_Score'] > threshold_periportal, 'zonation'] = 'Periportal'
    
    
    sc.pl.spatial(adata, color=["zonation"], color_map="Set2", size=1.5)

In [ ]:
# transgene_id = 'cisAAV-CMV-GFP-WPRE'
# df_results = pd.DataFrame(columns=['Sample_ID',  'condition', 'Gender', 'zonation', 'avg_expression'])


# for adata in adata_list:
    
#    if transgene_id in adata.var.index:
        
        # Step 1: Get coordinates of spots labeled as your category
#        coords = adata.obsm['spatial']
#        pericentral_coords = coords[adata.obs['zonation'] == 'Pericentral']
    
        # Step 2: Compute distance of each spot to the nearest labeled spot
#        distances = np.min(distance.cdist(coords, pericentral_coords, 'euclidean'), axis=1)
    
        # Step 3: Bin the distances (e.g., in 10-unit ranges)
        # bins = np.arange(0, distances.max(), 200)
        # adata.obs['distance_bin'] = pd.cut(distances, bins=bins)
    
#        current_sample = adata.obs['Sample_ID'].unique()[0]
#        current_condition = adata.obs['Condition'].unique()[0]
#        current_gender = adata.obs['Gender'].unique()[0]
        
#        print(current_sample)
#        print(current_condition)
#        print(current_gender)
        
#        sc.pl.violin(adata, keys=transgene_id, groupby='zonation', rotation=90)
        
        
        # Step 4: Compute average gene expression for each distance bin
#        gene_idx = adata.var_names.get_loc(transgene_id)
#        adata.obs[transgene_id] = adata.X[:, gene_idx].toarray().squeeze()
#        adata.obs['distance'] = distances
        
        # sns.scatterplot(data=adata.obs, x='distance', y=transgene_id)
        # plt.show()
#        sns.regplot(data=adata.obs, x='distance', y=transgene_id)
#        plt.show()
        # sns.violinplot(x="distance", y=transgene_id, data=adata.obs)
        # plt.show()
           
        
        # We store in a data frame some information for further visualization
        
 #       avg_expr_per_zonation = adata.obs.groupby('zonation')[transgene_id].mean()
        
#       for zonation, avg_expr in avg_expr_per_zonation.items():
#            df_results = df_results.append({
#                'Sample_ID': current_sample,
#                'condition': current_condition,
#                'Gender': current_gender,
#                'zonation': zonation,
#                'avg_expression': avg_expr
#            }, ignore_index=True)

In [ ]:
transgene_id = 'cisAAV-CMV-GFP-WPRE'
df_results = pd.DataFrame(columns=['Sample_ID', 'condition', 'Gender', 'zonation', 'avg_expression'])

rows_to_add = []  # List to collect rows

for adata in adata_list:
    
    if transgene_id in adata.var.index:
        
        # Get coordinates of spots labeled as your category
        coords = adata.obsm['spatial']
        pericentral_coords = coords[adata.obs['zonation'] == 'Pericentral']
    
        # Compute distance of each spot to the nearest labeled spot
        distances = np.min(distance.cdist(coords, pericentral_coords, 'euclidean'), axis=1)
    
        current_sample = adata.obs['Sample_ID'].unique()[0]
        current_condition = adata.obs['Condition'].unique()[0]
        current_gender = adata.obs['Gender'].unique()[0]
        
        print(current_sample)
        print(current_condition)
        print(current_gender)
        
        sc.pl.violin(adata, keys=transgene_id, groupby='zonation', rotation=90)
        
        # Compute average gene expression for each distance bin
        gene_idx = adata.var_names.get_loc(transgene_id)
        adata.obs[transgene_id] = adata.X[:, gene_idx].toarray().squeeze()
        adata.obs['distance'] = distances
        
        sns.regplot(data=adata.obs, x='distance', y=transgene_id)
        plt.show()
        
        # Store in a data frame some information for further visualization
        avg_expr_per_zonation = adata.obs.groupby('zonation')[transgene_id].mean()
        
        for zonation, avg_expr in avg_expr_per_zonation.items():
            # Instead of appending directly, we add to the list
            rows_to_add.append({
                'Sample_ID': current_sample,
                'condition': current_condition,
                'Gender': current_gender,
                'zonation': zonation,
                'avg_expression': avg_expr
            })

# After the loop, concatenate all collected rows
df_results = pd.concat([df_results, pd.DataFrame(rows_to_add)], ignore_index=True)

In [ ]:
df_results

In [ ]:
g = sns.FacetGrid(df_results, row="condition", col="Gender", height=4, aspect=1.5)
g.map(sns.violinplot, "zonation", "avg_expression", order=["Pericentral", "Periportal", "Other"])

In [ ]:
df_results

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

g = sns.catplot(data=df_results, x="zonation", y="avg_expression", hue="Gender", col="condition", kind="bar", height=4, aspect=1, errorbar='sd')
g.set_axis_labels("Zone", "Average Expression").set_titles("Condition {col_name}")

# Ensure correct order of conditions
condition_order = g.col_names  
grouped_df = df_results.groupby("condition")

# Perform statistical tests and annotate p-values at fixed y=3 in correct order
for ax, condition in zip(g.axes.flat, condition_order):  # Now matches plot order
    sub_df = grouped_df.get_group(condition)  # Get data for the correct condition

    for i, zonation in enumerate(sub_df["zonation"].unique()):
        subset = sub_df[sub_df["zonation"] == zonation]
        male_values = subset[subset["Gender"] == "Male"]["avg_expression"]
        female_values = subset[subset["Gender"] == "Female"]["avg_expression"]

        # Perform statistical test
        if len(male_values) > 1 and len(female_values) > 1:
            stat, p_value = ttest_ind(male_values, female_values, equal_var=False)  # Welch’s t-test
            # stat, p_value = mannwhitneyu(male_values, female_values, alternative="two-sided")  # Optional
        else:
            p_value = None

        # Annotate with p-value at fixed y=3
        if p_value is not None:
            text = f"p = {p_value:.3f}" if p_value >= 0.001 else "p < 0.001"
            ax.text(x=i,  # Changing along x-axis (zonation)
                    y=3,  # Fixed position at y=3
                    s=text, 
                    ha="center", fontsize=10, color="black", weight="bold")

plt.show()

In [ ]:
g = sns.relplot(data=df_results, x="zonation", y="avg_expression", hue="Gender", col="condition", kind="line", height=4, aspect=1, errorbar='sd')
g.set_axis_labels("Zone", "Average Expression").set_titles("Condition {col_name}")

plt.show()

In [ ]:
df_results_v2 = df_results
df_results_v2['zonation']=df_results_v2['zonation'].replace('Other', 'Intermediate')

In [ ]:
df_results_v2

In [ ]:
zonation_order  = ['Pericentral','Intermediate','Periportal']
df_results_v2["zonation"] = pd.Categorical(df_results_v2["zonation"], categories=zonation_order, ordered=True)

In [ ]:
g = sns.catplot(data=df_results_v2, x="zonation", y="avg_expression", hue="Gender", col="condition", kind="bar", height=4, aspect=1, errorbar='sd', order=zonation_order)
g.set_axis_labels("Zone", "Average Expression").set_titles("Condition {col_name}")

# Ensure correct order of conditions
condition_order = g.col_names  
grouped_df = df_results_v2.groupby("condition")

# Perform statistical tests and annotate p-values at fixed y=3 in correct order
for ax, condition in zip(g.axes.flat, condition_order):  # Now matches plot order
    sub_df = grouped_df.get_group(condition)  # Get data for the correct condition
    
    for i, zonation in enumerate(zonation_order):  # Use the correct zonation order
        subset = sub_df[sub_df["zonation"] == zonation]
        male_values = subset[subset["Gender"] == "Male"]["avg_expression"]
        female_values = subset[subset["Gender"] == "Female"]["avg_expression"]

        # Perform statistical test
        if len(male_values) > 1 and len(female_values) > 1:
            stat, p_value = ttest_ind(male_values, female_values, equal_var=False)  # Welch’s t-test
            # stat, p_value = mannwhitneyu(male_values, female_values, alternative="two-sided")  # Optional
        else:
            p_value = None

        # Annotate with p-value at fixed y=3
        if p_value is not None:
            text = f"p = {p_value:.3f}" if p_value >= 0.001 else "p < 0.001"
            ax.text(x=i,  # Changing along x-axis (zonation) in correct order
                    y=3.25,  # Fixed position at y=3
                    s=text, 
                    ha="center", fontsize=10, color="black", weight="bold")

plt.show()

In [ ]:
# for current_adata in adata_list:
#    current_sample = np.asarray(current_adata.obs["Sample_ID"].unique())
#    filename = 'adata_zonation_' + current_sample[0] + '.h5ad'
#    current_adata.write(os.path.join(results_folder, 'zonation' , filename))

In [ ]:
! jupyter nbconvert --to html 12_ZonationScores.ipynb